In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.8 MB/s eta 0:00:00


In [ ]:
import os
import re
import math
import time
import numpy as np
import pandas as pd
import pdfplumber
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LAParams
import json
from google import genai
from google.genai import types
from google.colab import drive
from google.colab import userdata
userdata.get('GEMINI_API_KEY')
drive.mount('/content/drive')

INPUT_DIR = "/content/drive/MyDrive/annual_reports"
OUTPUT_DIR = "extracted_letters"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
d_companies = {
    "Tesla": {"ticker": "TSLA", "exchange": "NASDAQ", "country": "US"},
    "BMW": {"ticker": "BMW.DE", "exchange": "Xetra", "country": "EU"},
    "Volkswagen": {"ticker": "VOW3.DE", "exchange": "Xetra", "country": "EU"},
    "Benz": {"ticker": "MBG.DE", "exchange": "Xetra", "country": "EU"},
    "Toyota": {"ticker": "7203.T", "exchange": "Tokyo", "country": "JP"},
    "Stellantis": {"ticker": "STLA", "exchange": "NYSE", "country": "EU"},
    "Bosch": {"ticker": "BOS.IN", "exchange": "IN", "country": "IN"},
    "Apple": {"ticker": "AAPL", "exchange": "NASDAQ", "country": "US"},
    "Microsoft": {"ticker": "MSFT", "exchange": "NASDAQ", "country": "US"},
    "Intel": {"ticker": "INTC", "exchange": "NASDAQ", "country": "US"},
    "Qualcomm": {"ticker": "QCOM", "exchange": "NASDAQ", "country": "US"},
    "Nvdia": {"ticker": "NVDA", "exchange": "NASDAQ", "country": "US"},
    "SAP": {"ticker": "SAP", "exchange": "NYSE / Xetra", "country": "EU"},
    "IBM": {"ticker": "IBM", "exchange": "NYSE", "country": "US"},
    "Jpmorgan": {"ticker": "JPM", "exchange": "NYSE", "country": "US"},
    "Goldman": {"ticker": "GS", "exchange": "NYSE", "country": "US"},
    "HSBC": {"ticker": "HSBC", "exchange": "NYSE", "country": "EU"},
    "Blackrock": {"ticker": "BLK", "exchange": "NYSE", "country": "US"},
    "Citigroup": {"ticker": "C", "exchange": "NYSE", "country": "US"},
    "Pfizer": {"ticker": "PFE", "exchange": "NYSE", "country": "US"},
    "J&J": {"ticker": "JNJ", "exchange": "NYSE", "country": "US"},
    "Nestle": {"ticker": "NESN.SW", "exchange": "Swiss", "country": "EU"},
    "Loreal": {"ticker": "OR.PA", "exchange": "Paris", "country": "EU"},
    "Shiseido": {"ticker": "4911.T", "exchange": "Tokyo", "country": "JP"},
    "P&G": {"ticker": "PG", "exchange": "NYSE", "country": "US"}
}

def get_slug(name, data):
    if name == "Bosch": return None
    # Manual Overrides
    if name == "BMW": return "OTC_BAMGF"
    if name == "Volkswagen": return "OTC_VWAGY"
    if name == "Benz": return "OTC_MBGAF"
    if name == "Nestle": return "OTC_NSRGY"
    if name == "Toyota": return "NYSE_TM"
    if name == "Loreal": return "Loreal_SA"
    if name == "Shiseido": return "Shiseido_Company_Limited"

    # Standard "EXCHANGE_TICKER" logic
    exchange = data['exchange']
    ticker = data['ticker']
    if "NYSE" in exchange: return f"NYSE_{ticker}"
    if "NASDAQ" in exchange: return f"NASDAQ_{ticker}"
    return None

In [ ]:
START_PATTERNS = [
    r"to\s+our\s+shareholders",
    r"dear\s+shareholders",
    r"letter\s+from\s+the",
    r"message\s+from",
    r"executive\s+statement",
    r"chairman['’]s\s+statement",
    r"chief\s+executive['’]s\s+review",
    r"my\s+fellow\s+shareholders",
    r"letter\s+from\s+your\s+ceo",
    r"letter\s+from\s+your\s+chairman",
    r"dear\s+fellow\s+shareholders",
    r"dear\s+fellow",
    r"dear\s+shareowners",
    r"dear\s+stakeholders",
    r"dear\s+stakeholder",
    r"dear\s+nvidians"
]

STOP_PATTERNS = [
    r"sincerely",
    r"yours\s+truly",
    r"financial\s+review",        # Often the next section
    r"management['’]s\s+discussion", # MD&A section often follows
    r"consolidated\s+financial"   # Financial tables often follow
]

def extract_letter_from_pdf(pdf_path):
    extracted_text = ""
    start_found = False

    # We still use a max limit to prevent grabbing the whole book if we miss the stop signal
    max_pages_limit = 10
    pages_grabbed = 0

    # LAParams(detect_vertical=True) helps to detect text columns
    laparams = LAParams()

    try:
        # extract_pages performs layout analysis to group text into columns/boxes
        # We limit to first 20 pages using page_numbers
        page_iter = extract_pages(pdf_path, page_numbers=range(20), laparams=laparams)

        for i, page_layout in enumerate(page_iter):
            text = ""
            # iterate through layout elements (text boxes, figures, etc.)
            for element in page_layout:
                if isinstance(element, LTTextContainer):
                    text += element.get_text()

            if not text: continue

            clean_text = text.lower()

            # 1. SEARCH FOR START
            if not start_found:
                # Split page into lines to avoid matching Table of Contents entries (e.g., "Letter ... 5")
                lines = clean_text.split('\n')
                for j, line in enumerate(lines):
                    for pattern in START_PATTERNS:
                        if re.search(pattern, line):
                            # Check 1: Line itself ends in a small number (TOC entry)
                            toc_match = re.search(r'(\d+)\s*$', line.strip())
                            if toc_match:
                                number = int(toc_match.group(1))
                                if number < 1900: # Avoid skipping headers with years like "Review 2023"
                                    print(f"  [SKIP] Ignoring TOC-like line '{line.strip()}' on page {i+1}")
                                    continue

                            # Check 2: Next couple of lines START with page numbers (Multi-line TOC)
                            # e.g. "Letter to Shareholders" \n "5 ..." or "5 Overview"
                            is_multiline_toc = False
                            for offset in range(1, 5): # Look ahead 4 lines
                                if j + offset < len(lines):
                                    next_line_check = lines[j+offset].strip()

                                    # Check for number at start of line (possibly preceded by dots/spaces)
                                    # or "page X"
                                    match_start = re.match(r'^[\.\s]*(\d+)', next_line_check)
                                    match_page = re.match(r'^page\s+(\d+)', next_line_check)

                                    found_num = None
                                    if match_start:
                                        found_num = int(match_start.group(1))
                                    elif match_page:
                                        found_num = int(match_page.group(1))

                                    if found_num is not None and found_num < 1900:
                                         # FIX: Ignore if the line looks like a percentage (e.g. "61 percent...")
                                         if "percent" in next_line_check.lower() or "%" in next_line_check:
                                             continue

                                         print(f"  [SKIP] Ignoring TOC-like header '{line.strip()}' followed by TOC-like line '{next_line_check}' on page {i+1}")
                                         is_multiline_toc = True
                                         break

                            if is_multiline_toc:
                                continue

                            # Check 3: Line starts with a small number (TOC entry with leading page num)
                            # e.g. "5 ... Letter to Shareholders"
                            start_num_match = re.match(r'^\s*(\d+)', line.strip())
                            if start_num_match:
                                number = int(start_num_match.group(1))
                                if number < 1900:
                                    print(f"  [SKIP] Ignoring TOC-like line (leading num) '{line.strip()}' on page {i+1}")
                                    continue

                            # Check 4: Previous line is just a page number (Multi-line TOC with leading page num)
                            # e.g. "5" \n "Letter to Shareholders"
                            if j > 0:
                                prev_line = lines[j-1].strip()
                                if re.match(r'^[\.\s]*\d+$', prev_line) or re.match(r'^page\s+\d+$', prev_line):
                                     num_groups = re.findall(r'\d+', prev_line)
                                     if num_groups and int(num_groups[-1]) < 1900:
                                         print(f"  [SKIP] Ignoring TOC-like header '{line.strip()}' preceded by '{prev_line}' on page {i+1}")
                                         continue

                            print(f"  [START] Found start signature '{pattern}' on page {i+1}")
                            start_found = True
                            break
                    if start_found: break

            # 2. EXTRACT & SEARCH FOR STOP
            if start_found:
                extracted_text += f"\n--- Page {i+1} ---\n"
                extracted_text += text
                pages_grabbed += 1

                # Check for STOP signals in this page
                for stop_pat in STOP_PATTERNS:
                    if re.search(stop_pat, clean_text):
                        print(f"  [STOP] Found end signature '{stop_pat}' on page {i+1}")
                        return extracted_text # <--- STOP HERE IMMEDIATELY

                # Emergency Brake: If we grabbed too many pages, stop anyway
                if pages_grabbed >= max_pages_limit:
                    print("  [LIMIT] Reached max page limit.")
                    return extracted_text

        return extracted_text

    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Walk through the directory structure
for root, dirs, files in os.walk(INPUT_DIR):
    for file in files:
        if file.endswith(".pdf"):
            company_name = os.path.basename(root)

            if company_name != "Nvdia":
                continue

            pdf_path = os.path.join(root, file)
            year = file.split("_")[-1].replace(".pdf", "") # naive year extractor

            print(f"Scanning {company_name} - {year}...")

            letter_text = extract_letter_from_pdf(pdf_path)

            if letter_text:
                # Save the extracted text
                output_filename = f"{company_name}_{year}_Letter.txt"
                output_path = os.path.join(OUTPUT_DIR, output_filename)

                with open(output_path, "w", encoding="utf-8") as f:
                    f.write(letter_text)
                print(f"  [SAVED] {output_filename}")
            else:
                print(f"  [MISSING] No letter pattern found in first 20 pages.")

Scanning Nvdia - 2012...
  [START] Found start signature 'dear\s+fellow' on page 3
  [STOP] Found end signature 'sincerely' on page 5
  [SAVED] Nvdia_2012_Letter.txt
Scanning Nvdia - 2011...
  [START] Found start signature 'dear\s+fellow' on page 12
  [STOP] Found end signature 'sincerely' on page 17
  [SAVED] Nvdia_2011_Letter.txt
Scanning Nvdia - 2010...
  [MISSING] No letter pattern found in first 20 pages.
Scanning Nvdia - 2019...
  [START] Found start signature 'dear\s+nvidians' on page 12
  [SAVED] Nvdia_2019_Letter.txt
Scanning Nvdia - 2014...
  [START] Found start signature 'dear\s+stakeholders' on page 12
  [STOP] Found end signature 'sincerely' on page 14
  [SAVED] Nvdia_2014_Letter.txt
Scanning Nvdia - 2017...
  [START] Found start signature 'dear\s+nvidians' on page 12
  [SAVED] Nvdia_2017_Letter.txt
Scanning Nvdia - 2018...
  [START] Found start signature 'dear\s+nvidians' on page 12
  [SAVED] Nvdia_2018_Letter.txt
Scanning Nvdia - 2015...
  [START] Found start signature '

In [ ]:
print(f"Checking files in {OUTPUT_DIR} for keywords 'China' or 'Chinese'...")

letters_china = []

for root, dirs, files in os.walk(OUTPUT_DIR):
    for file in files:
        file_path = os.path.join(root, file)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                content = f.read()

            # Find matches with context using regex
            matches = list(re.finditer(r'(china|chinese)', content, re.IGNORECASE))

            if matches:
                letters_china.append(file)
                print(f"\n[MATCH] {file}")
                # Print up to 3 snippets per file
                for i, m in enumerate(matches[:3]):
                    start = max(0, m.start() - 50)
                    end = min(len(content), m.end() + 50)
                    snippet = content[start:end].replace('\n', ' ')
                    print(f"   Snippet {i+1}: ...{snippet}...")
                if len(matches) > 3:
                    print(f"   ... and {len(matches) - 3} more matches.")

        except Exception as e:
            # print(f"[ERROR] Could not read {file}: {e}")
            pass

print(f"\nFound {len(letters_china)} files containing keywords.")

# Create the new directory
CHINA_DIR = "extracted_letters_china"
if not os.path.exists(CHINA_DIR):
    os.makedirs(CHINA_DIR)

print(f"\nCopying files to {CHINA_DIR}...")
for file in letters_china:
    src_path = os.path.join(OUTPUT_DIR, file)
    dst_path = os.path.join(CHINA_DIR, file)

    try:
        with open(src_path, "r", encoding="utf-8") as f_src:
            text_content = f_src.read()

        with open(dst_path, "w", encoding="utf-8") as f_dst:
            f_dst.write(text_content)
    except Exception as e:
        print(f"Error copying {file}: {e}")

print("Copying complete.")

Checking files in extracted_letters for keywords 'China' or 'Chinese'...

[MATCH] Stellantis_2019_Letter.txt
   Snippet 1: ...ep family, which also represented our  entry into China’s rapidly-growing New Energy Vehicle market.  We ...

[MATCH] Nestle_2022_Letter.txt
   Snippet 1: ...to shop in six months – with new sites  in Chile, China, India and Arlington, Virginia, in the  United St...
   Snippet 2: ...ment. Through our regional R&D centers in Africa, China, India  and South-East Asia, we can innovate clos...
   Snippet 3: ...rther refining for launch. Using these tools,  in China we launched Nescafé Dalgona coffee mixes  and Nes...
   ... and 4 more matches.

[MATCH] Blackrock_2010_Letter.txt
   Snippet 1: ...o plague the U.S. housing market, home  prices in China soared amid worries of new asset bubbles in the f...
   Snippet 2: ...tainty for markets.  in the standard of living in China have spurred  Debtor nations in the euro zone are...
   Snippet 3: ...zone are facing tremen- 

In [ ]:
data_rows = []

for filename in letters_china:
    # Expected format: "CompanyName_Year_Letter.txt"
    # using rsplit to safely handle names with underscores if any (though logic suggests single word or simple names)
    # based on previous logic: company_name = filename.split("_")[0]
    parts = filename.split("_")

    # Handle cases where company name might contain underscores?
    try:
        base = filename.replace("_Letter.txt", "")
        # finding the last underscore to separate company and year
        last_underscore_index = base.rfind("_")
        if last_underscore_index == -1:
            continue

        company_name = base[:last_underscore_index]
        year = base[last_underscore_index+1:]

        # Get data from dictionary
        if company_name in d_companies:
            company_data = d_companies[company_name]
            slug = get_slug(company_name, company_data)

            if slug:
                # Reconstruct URL logic
                clean_slug = slug.replace("NASDAQ_", "").replace("NYSE_", "").replace("OTC_", "")
                first_letter = clean_slug[0].lower()
                pdf_filename = f"{slug}_{year}.pdf"
                url = f"https://www.annualreports.com/HostedData/AnnualReportArchive/{first_letter}/{pdf_filename}"

                # Read the text content
                file_path = os.path.join(CHINA_DIR, filename)
                with open(file_path, "r", encoding="utf-8") as f:
                    letter_text = f.read()

                data_rows.append({
                    "Company Name": company_name,
                    "Year": year,
                    "url": url,
                    "Letter Text": letter_text
                })
        else:
            print(f"Warning: {company_name} not found in d_companies")

    except Exception as e:
        print(f"Error parsing {filename}: {e}")

df = pd.DataFrame(data_rows)
display(df)

,Company Name,Year,url,Letter Text
0,Stellantis,2019,https://www.annualreports.com/HostedData/Annua...,\n--- Page 9 ---\n7 \nMessage from the Chairma...
1,Nestle,2022,https://www.annualreports.com/HostedData/Annua...,\n--- Page 6 ---\nStaying the course \nDear fe...
2,Blackrock,2010,https://www.annualreports.com/HostedData/Annua...,\n--- Page 6 ---\nMy Fellow Shareholders\nApri...
3,Blackrock,2014,https://www.annualreports.com/HostedData/Annua...,\n--- Page 17 ---\nl auRence d. fInk Chairma...
4,Pfizer,2012,https://www.annualreports.com/HostedData/Annua...,\n--- Page 8 ---\nFinancial Review\nPfizer Inc...
...,...,...,...,...
72,Nestle,2017,https://www.annualreports.com/HostedData/Annua...,"\n--- Page 6 ---\nDear fellow shareholders,\nF..."
73,BMW,2014,https://www.annualreports.com/HostedData/Annua...,\n--- Page 7 ---\n7 REPORT OF THE SUPERVISOR...
74,Citigroup,2017,https://www.annualreports.com/HostedData/Annua...,\n--- Page 4 ---\nLetter To Shareholders\n7\n1...
75,Goldman,2014,https://www.annualreports.com/HostedData/Annua...,\n--- Page 2 ---\nThe Goldman Sachs Business P...


In [ ]:
client = genai.Client(api_key=userdata.get('GEMINI_API_KEY'))
MODEL_NAME = "gemini-2.5-flash"

In [ ]:
BATCH_SIZE = 5
analysis_results = {} # Dictionary to store results by index

if df.empty:
    print("Warning: DataFrame is empty.")
else:
    total_batches = (len(df) + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"Processing {len(df)} letters in {total_batches} batches (Batch Size: {BATCH_SIZE})...")

    # Iterate through the DataFrame in chunks
    for batch_idx, i in enumerate(range(0, len(df), BATCH_SIZE)):
        batch = df.iloc[i : i + BATCH_SIZE]

        # Construct a combined prompt for the batch
        prompt_text = """
        You are an expert financial analyst. Analyze the following shareholder letters regarding their discussion of China.

        For EACH letter provided below (identified by LETTER_ID), perform the following:
        1. Identify the author's name and role.
        2. Summarize the discussion regarding "China".
        3. Determine the sentiment towards China (optimistic growth, cautious engagement, hedging, de-risking, uncertainty management, etc, etc).

        Output strict JSON. Return a list of objects. Each object MUST include the 'letter_id' (integer) provided in the header.
        Format:
        [
            {"letter_id": <int>, "author_name": "...", "author_role": "...", "china_summary": "...", "china_sentiment": "..."},
            ...
        ]
        """

        # Append text of each letter in the batch
        for idx, row in batch.iterrows():
            # Truncate text to ensure we fit multiple letters in context (approx 20k chars per letter is safe for Flash)
            safe_text = row['Letter Text'][:20000]
            prompt_text += f"\n\n=== START LETTER_ID {idx} ({row['Company Name']} {row['Year']}) ===\n{safe_text}\n=== END LETTER_ID {idx} ==="

        # Retry logic for the batch
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = client.models.generate_content(
                    model=MODEL_NAME,
                    contents=[types.Content(parts=[types.Part.from_text(text=prompt_text)])],
                    config=types.GenerateContentConfig(response_mime_type="application/json")
                )

                # Parse response
                batch_json = json.loads(response.text)

                # Store results mapped by letter_id
                if isinstance(batch_json, list):
                    for item in batch_json:
                        lid = item.get('letter_id')
                        if lid is not None:
                            analysis_results[lid] = item

                print(f"Batch {batch_idx+1}/{total_batches} completed.")
                time.sleep(2) # Respect rate limits
                break
            except Exception as e:
                print(f"Batch {batch_idx+1} attempt {attempt+1} failed: {e}")
                time.sleep(5)

    # Merge results back into the DataFrame
    if analysis_results:
        # Create DataFrame from results dict (keys are indices)
        results_df = pd.DataFrame.from_dict(analysis_results, orient='index')

        # Select only the columns we want to add
        cols_to_add = ['author_name', 'author_role', 'china_summary', 'china_sentiment']

        # Ensure columns exist in results (in case of empty results or partial failures)
        for col in cols_to_add:
            if col not in results_df.columns:
                results_df[col] = None

        # Join with original DataFrame
        df_final = df.join(results_df[cols_to_add])
        display(df_final.head())
    else:
        print("No results generated.")

Processing 77 letters in 16 batches (Batch Size: 5)...
Batch 1/16 completed.
Batch 2/16 completed.
Batch 3/16 completed.
Batch 4/16 completed.
Batch 5/16 completed.
Batch 6/16 completed.
Batch 7/16 completed.
Batch 8/16 completed.
Batch 9/16 completed.
Batch 10/16 completed.
Batch 11/16 completed.
Batch 12/16 completed.
Batch 13/16 completed.
Batch 14 attempt 1 failed: Expecting ',' delimiter: line 6 column 60 (char 210)
Batch 14/16 completed.
Batch 15/16 completed.
Batch 16/16 completed.


,Company Name,Year,url,Letter Text,author_name,author_role,china_summary,china_sentiment
0,Stellantis,2019,https://www.annualreports.com/HostedData/Annua...,\n--- Page 9 ---\n7 \nMessage from the Chairma...,John Elkann; Mike Manley,Chairman; Chief Executive Officer,"Fiat Chrysler Automobiles (FCA), now Stellanti...",Cautious engagement. While acknowledging 'cont...
1,Nestle,2022,https://www.annualreports.com/HostedData/Annua...,\n--- Page 6 ---\nStaying the course \nDear fe...,Paul Bulcke; U. Mark Schneider,Chairman; Chief Executive Officer,The letter does not specifically discuss China...,Not explicitly stated. The general sentiment t...
2,Blackrock,2010,https://www.annualreports.com/HostedData/Annua...,\n--- Page 6 ---\nMy Fellow Shareholders\nApri...,Laurence D. Fink,Chairman & Chief Executive Officer,BlackRock observed soaring home prices in Chin...,Cautious engagement. While recognizing China a...
3,Blackrock,2014,https://www.annualreports.com/HostedData/Annua...,\n--- Page 17 ---\nl auRence d. fInk Chairma...,Laurence D. Fink,Chairman and Chief Executive Officer,BlackRock cited China's Kuaidi as an example o...,Optimistic engagement. China is presented in a...
4,Pfizer,2012,https://www.annualreports.com/HostedData/Annua...,\n--- Page 8 ---\nFinancial Review\nPfizer Inc...,Not explicitly stated (Pfizer Inc. and Subsidi...,Company Management,Pfizer stated that it operates in multiple for...,Neutral/operational. The discussion of China i...


In [ ]:
df_final["source"] = "Shareholder Letter"

In [ ]:
df_final.to_csv("extracted_letters_china_gemini.csv", index=False)